In [ ]:
import os
import sys
import json
from Visualisation import PlotlyViz
import operator
from ArchiveAnalysis import PersonAnalyser
from datetime import datetime, timedelta, timezone
import csv
import locale
import pandas as pd
import copy

In [ ]:
viz = PlotlyViz.PlotlyViz("OFFLINE")

In [ ]:
def formatName(personName):
    if "," in personName:
        parts = personName.split(",")
        return "%s %s"%(parts[1].strip(), parts[0].strip())

In [ ]:
def convertDateFormat(date):
    dateParts = date.split("-")
    return "%s-%s-%s"%(dateParts[2], dateParts[1], dateParts[0])

In [ ]:
def perdelta(start, end, delta):
    curr = start
    while curr <= end:
        yield curr
        curr += delta

In [ ]:
def plotAppearancesPerCategoryValue(category, categoryLabel):
    categoryValues, counts = personAnalyser.countAppearancesPerColumnValue(category, "Date", sortColumn="Date")
    viz.plotYAgainstXAsBarChart(
        categoryValues, 
        counts, "Number of appearances per %s"%categoryLabel, categoryLabel.capitalize(), "Number of appearances", {}, "appearances-per-%s"%categoryLabel)  
    return categoryValues, counts

In [ ]:
def plotAveragesPerCategoryValue(valueLabel, category, categoryLabel):
    categoryValues, averages = personAnalyser.calculateAverageTimePerColumnValue(category, [valueLabel], sortColumn=valueLabel)
    viz.plotYAgainstXAsBarChart(
    categoryValues, 
    averages[0], "Average %s per %s"%(valueLabel, categoryLabel), categoryLabel.capitalize(), "Average %s"%(valueLabel), {}, "average-%s-per-%s"%(valueLabel, categoryLabel))  
    return categoryValues, averages[0]

In [ ]:
def plotTotalsPerCategoryValue(valueLabel, category, categoryLabel):
    categoryValues, totals = personAnalyser.calculateTotalTimePerColumnValue(category, [valueLabel], sortColumn=valueLabel)
    viz.plotYAgainstXAsBarChart(
    categoryValues, 
    totals[0], "Total %s per %s"%(valueLabel, categoryLabel), categoryLabel.capitalize(), "Total %s"%(valueLabel), {}, "total-%s-per-%s"%(valueLabel, categoryLabel))  
    return categoryValues, totals[0]

In [ ]:
appearances = pd.read_csv("appearances.csv", sep=';')
personAnalyser = PersonAnalyser.PersonAnalyser(appearances)
totalNumberOfAppearances = personAnalyser.countRowsInDataframe()
totalFaceTime = personAnalyser.getColumnTotal("Time face recognised (s)")
totalSpeakerTime = personAnalyser.getColumnTotal("Time voice recognised (s)")
totalTime = personAnalyser.getColumnTotal("Total time recognised (s)")
print(appearances)

## Number of appearances by a politician

In [ ]:
category = "Name"
categoryLabel = "person"
names, appearanceCounts = plotAppearancesPerCategoryValue(category, categoryLabel)

## Length of appearance time for politician

In [ ]:
keysList, valuesLists = personAnalyser.calculateTimeBreakdownPerColumnValue("Name", ["Time face recognised (s)", "Time voice recognised (s)"], sortColumn="Total time recognised (s)")
namesList = ["Face", "Speaker"]
keysLists = [keysList, keysList]
viz.plotStackedBarChart(valuesLists, keysLists,  namesList, "Time appeared (s) for all politicians", "Person", "Time appeared (s)", "person-appearance-time-all-politicians", {})



## Number of appearances per gender

In [ ]:
category = "Gender"
categoryLabel = "gender"
genders, counts = plotAppearancesPerCategoryValue(category, categoryLabel)
maleAppearanceCounts = counts[genders.index("M")]   
femaleAppearanceCounts = counts[genders.index("V")]  

## Average onscreen time per gender

In [ ]:
valueLabel = "Time face recognised (s)"
category = "Gender"
categoryLabel = "Gender"
genders, averages = plotAveragesPerCategoryValue(valueLabel, category, categoryLabel)


## Average speaking time per gender

In [ ]:
valueLabel = "Time voice recognised (s)"
category = "Gender"
categoryLabel = "Gender"
genders, averages = plotAveragesPerCategoryValue(valueLabel, category, categoryLabel)


## Speaking time along the political spectrum

In [ ]:
valueLabel = "Time voice recognised (s)"
category = categoryLabel = "Party"
parties, partyTimes = plotTotalsPerCategoryValue(valueLabel, category, categoryLabel)


In [ ]:
valueLabel = "Time voice recognised (s)"
category = categoryLabel = "Party ideology"
ideologies, ideologyTimes = plotTotalsPerCategoryValue(valueLabel, category, categoryLabel)
  